## AlexNet

Ref: <a href ='https://proceedings.neurips.cc/paper_files/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf'>ImageNet Classification with Deep Convolutional Neural Networks</a>

AlexNet은 ILSVRC-2012에서 기존 대회에서 강세를 보이던 머신러닝 기반 모델들을 제치고 우승을 차지한 딥러닝 기반 모델이다. CNN 기반 구조를 가지고 있는 AlexNet은 CNN의 대부흥을 이끌었으며 현재까지 10만번 이상 인용된 기념비적인 논문이라고 할 수 있다. AlexNet은 2개의 GPU로 병렬연산을 수행하기 위해 병렬적인 구조로 설계되었으며 5개의 컨볼루션 레이어와 3개의 FC 레이어로 이루어져있다. 여기서 독특한 점은 세 번째 컨볼루션 레이어는 다른 컨볼루션 레이어들과 달리 이전 단계의 두 채널의 특성맵들과 모두 연결되어 있다는 것이다.

$227\times227\times3$ 사이즈의 이미지를 입력으로 받는 구조로, 학습 데이터셋의 평균 픽셀 값을 빼는 방식으로 정규화를 진행한다. 두 번째와 세 번째 컨볼루션 레이어 이전에 MaxPooling이 수행되며 마지막 컨볼루션 레이어 뒤에도 MaxPool이 들어간다.

첫 번째 컨볼루션 레이어는 커널 사이즈 11, stride 4, 그리고 out channel 96의 구조를 가지며 zero-padding은 수행하지 않았다. 이후 ReLU 활성화 함수를 거친 후 $3\times3$ overlapping MaxPool이 stride 2로 시행된다. 여기에 Local Response Normalization(LRN)을 적용하여 수렴 속도를 높인다. 이 레이어의 출력 특성맵은 96개 채널을 가지는데 이를 두 개의 GPU로 분산하여 각각 48개의 특성맵을 처리하게 된다.

위와 비슷한 방법으로 다섯 번째 컨볼루션 레이어까지 진행하고 나면 $6\times6\times256$의 특성맵을 얻을 수 있는데 이를 두 개의 FC 레이어 + ReLU와 한 개의 FC 레이어 + softmax를 거쳐 클래스별 확률을 계산하게 된다. 

#### ReLU
AlexNet은 활성화 함수로 기존의 tanh 함수 대신 ReLU(Rectified Linear Unit)를 사용했다. 이는 tanh보다 6배 빠른 수렴 속도를 보이면서 유사한 정확도를 보였다고 한다. 이는 이후 Dying ReLU 문제가 대두되기 전까지 가장 많이 사용되었으며 현재까지도 종종 사용되는 활성화 함수이다.

#### Drop-out
과적합(over-fitting)을 막기 위해 Drop-out 규제를 사용했다. 이는 FC 레이어의 뉴런 중 일부를 랜덤하게 0으로 만드는 것으로 값이 0이 된 뉴런은 forward pass와 back propagation 단계를 통한 가중치 업데이트에 영향을 미치지 않게 된다. Drop-out은 학습 시 사용되는 기법으로, 테스트 시에는 적용하지 않는다.

#### Overlapping Pooling
CNN에서는 컨볼루션 연산을 통해 얻은 특성 맵의 크기를 줄이기 위해 Pooling을 사용한다. 이때 Pooling kernel이 움직이는 보폭인 stride를 kernel size보다 작게하면 겹치는 부분이 생기는데 이것을 Ovelapping pooling이라고 한다. AlexNet은 Overlapping MaxPool을 사용했다. 이를 사용하면 top-k error를 줄이는데 효과가 있다고 한다.

#### Local Response Normalization (LRN)
AlexNet은 활성화된 뉴런이 주변 이웃 뉴런의 활성을 억제하는, 신경생물학에서의 Lateral inhibitation의 개념에서 착안하여, 강하게 활성화된 뉴런의 주변 뉴런을 Normalization 하는 방식을 적용했다.

$$b^i_{x,y} = a^i_{x,y} / (k+\alpha \sum^{min(N-1, i+n/2)}_{j=max(0,i-n/2)}(a^j_{x,y})^2)^\beta$$

#### Data Augmentation
AlexNet은 사진 내 랜덤한 위치로부터 더 작은 사이즈의 이미지를 crop 하는 RandomCrop과, 이미지의 좌우를 반전시키는 Horizontal reflection을 통해 데이터를 증강하였고 이는 과적합 방지 및 일반화 성능의 증가로 이어졌다.

In [11]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

from torchvision import datasets, transforms

In [12]:
class AlexNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=2, padding=0)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.lrn = nn.LocalResponseNorm(size=5)

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(in_features=256*6*6, out_features=4096),
            nn.Dropout(),
            nn.Linear(in_features=4096, out_features=4096),
            nn.Linear(in_features=4096, out_features=1000)
        )
        
    def forward(self, x: torch.tensor):
        x = self.lrn(F.relu(self.conv1(x), inplace=True))
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        x = self.lrn(F.relu(self.conv2(x), inplace=True))
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        x = F.relu(self.conv3(x), inplace=True)
        x = F.relu(self.conv4(x), inplace=True)
        x = F.relu(self.conv5(x), inplace=True)
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        x = self.classifier(x)
        return x

In [13]:
data_transform = {
    'train': transforms.Compose(
        [
            transforms.RandomCrop(227), # train 단계에서는 random crop
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.490, 0.449, 0.411], [0.231, 0.221, 0.230])
        ]
    ),
    'val': transforms.Compose(
        [
            transforms.Resize(256), 
            transforms.CenterCrop(227), # valid 단계에서는 resize & center crop
            transforms.ToTensor(),
            transforms.Normalize([0.490, 0.449, 0.411], [0.231, 0.221, 0.230])
        ]
    )
}